In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from src.utils.trust_region import TrustRegion
from src.utils.lagrange_polynomial import LagrangePolynomials
import matplotlib.pyplot as plt
import numpy as np
import casadi as ca

# def myfunc(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
#     return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100
def myfunc(x:np.ndarray) -> np.ndarray:    
    return 10*(x[0]**2)*(1 + 0.75*np.cos(70*x[0])/12) + np.cos(100*x[0])**2/24 + 2*(x[1]**2)*(1 + 0.75*np.cos(70*x[1])/12) + np.cos(100*x[1])**2/24 + 4*x[0]*x[1]

tr_list = []
initial_centers = [np.array([[2.2], [2.12]]), 
                np.array([[-2.2], [-2.12]]),
                np.array([[-2.2], [2.12]]),
                np.array([[2.2], [-2.12]])]

for center in initial_centers:
    dataset = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [-1.0, 0.0], [1.0, 1.0], [0.0, -1.0]]).T + center
    INPUT_SYMBOL = ca.SX.sym('x', dataset.shape[0])

    poly_mf = LagrangePolynomials(input_symbols=INPUT_SYMBOL, pdegree=2)
    poly_mceq = LagrangePolynomials(input_symbols=INPUT_SYMBOL, pdegree=2)
    mf = []
    for i in range(dataset.shape[1]):
        x = dataset[:, i]
        mf.append(myfunc(x))
    mf = np.array(mf)
    
    sort_ind = mf.argsort(axis=0)  
    dataset = dataset[:, sort_ind]
    mf = mf[sort_ind]

    tr = TrustRegion(input_symbols=INPUT_SYMBOL, dataset=dataset, results=mf, func=myfunc)
    tr.run(max_radius=1.5, max_iter=30, rad_tol=1E-8)
    
    tr_list.append(tr)

In [ ]:
fsize = 15
tsize = 18
tdir = 'in'
major = 5.0
minor = 3.0

style = 'default'

plt.style.use(style)
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

fig, ax = plt.subplots(1)
for i, tr in enumerate(tr_list):
    ## Plot of objective function
    ax.plot(range(len(tr.list_of_OF)), tr.list_of_OF, label=f"dataset-{i+1}", linestyle='dashed')
    try:
        ax.set_yscale("log")
    except:
        ax.set_yscale("linear")
    ax.set_xlabel(f"Number of iterations")
    ax.set_ylabel(f"OF value")

plt.legend()
plt.savefig(f"./plots/score_OF_all.png")
plt.close()

In [ ]:
from matplotlib.colors import LogNorm
from matplotlib import ticker, cm
from casadi import *
import glob

x, y = np.meshgrid(np.linspace(-4, 4, 100),
                    np.linspace(-4, 4, 100))
    
# levels = list(np.arange(0, 5, 1))
# levels = [0.01, 0.05, 0.1, 0.5, 1, 5, 10]
# levels = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50]
levels = [10**i for i in np.arange(-1,2.5,0.2)]

old_center = [0,0]
old_radius = 0
    
files = glob.glob('./plots/*')
for f in files:
    os.remove(f)
tr = tr_list[0]
        
for i, m in enumerate(tr.list_of_models):
    if i % 1 == 0:
        center = m.sample_set.ball.center
        radius = tr.list_of_radius[i]
        intx = x - center[0]
        inty = y - center[1]
        dist = np.sqrt(intx**2 + inty**2)
        intindices = dist <= radius

        fig, ax = plt.subplots(1)
#         ax.set_title(f"{tr.list_of_status[i]['criticality_step']} \n {tr.list_of_status[i]['acceptance']}")
        ax.set_xlabel(f"$x_1$")
        ax.set_ylabel(f"$x_2$")
        func = myfunc([x, y])
        func_map = m.model_polynomial.feval.map(x.shape[0]*x.shape[1])
        x_dm = DM(x).reshape((1, x.shape[0]*x.shape[1]))
        y_dm = DM(y).reshape((1, x.shape[0]*x.shape[1]))
        # print(func_map, vertcat(x_dm, y_dm).shape)
        func_dm = func_map(vertcat(x_dm, y_dm))
        func_dm = func_dm.reshape((x.shape[0], x.shape[1])).full()
        func[intindices] = func_dm[intindices]
        
        
        circle1 = plt.Circle(center, radius, color='black', fill=False)
        circle2 = plt.Circle(old_center, old_radius, color='red', fill=False)
        CS = ax.contour(x, y, func, levels, norm = LogNorm(), cmap=cm.PuBu_r, locator=ticker.LogLocator())
        ax.clabel(CS, fontsize=9, inline=True)
        ax.add_patch(circle1)
        ax.add_patch(circle2)
        plt.scatter(m.y[0, :], m.y[1, :], color='black', label=f'iteration_{i}')
        plt.scatter(m.y[0, 0], m.y[1, 0], color='red', label=f'Best point')
        plt.legend()
        plt.savefig(f"./plots/TR_plots_{i}.png")
        plt.close()

        old_center = center*1
        old_radius = radius*1

        
## Plot of objective function
fig, ax = plt.subplots(1)
ax.set_title(f"Objective Function vs Iterations")
ax.scatter(range(len(tr.list_of_OF)), tr.list_of_OF, color="black", label="OF")
try:
    ax.set_yscale("log")
except:
    ax.set_yscale("linear")

plt.legend()
plt.savefig(f"./plots/score_OF.png")
plt.close()